In [ ]:
import xarray as xr
import fsspec
import numpy as np

Open Zarr datasets in Xarray using a mapper from fsspec.  We use `anon=True` for free-access public buckets like the AWS Open Data Program, and `requester_pays=True` for requester-pays public buckets. 

In [ ]:
url = 's3://noaa-nwm-retro-v2-zarr-pds'

In [ ]:
fs = fsspec.filesystem('s3', anon=False, profile='esip-qhub')

In [ ]:
%%time
ds = xr.open_zarr(fs.get_mapper(url), consolidated=True)

In [ ]:
ds

In [ ]:
var='streamflow'

In [ ]:
print(f'Variable size: {ds[var].nbytes/1e12:.1f} TB')

In [ ]:
idx = (ds.latitude > 41.0) & (ds.latitude < 51.0) & (ds.longitude > -75.0) & (ds.longitude < -62.0)

In [ ]:
ds_subset = ds.isel(feature_id=idx)

Test writing just the 1st time chunk:

In [ ]:
ds_out = ds_subset[[var]].isel(time=slice(0,672))

In [ ]:
ds_out = ds_subset[[var]].sel(time=slice('2000-01-01',None))

In [ ]:
ds_out

In [ ]:
print(f'Variable size: {ds_out.nbytes/1e9:.1f} GB')

In [ ]:
ds_out['streamflow'].encoding

In [ ]:
len(ds_out['feature_id'])

In [ ]:
encoding={}
for var in ds_out.data_vars:
    encoding[var] = dict(zlib=True, complevel=5, 
                         fletcher32=False, shuffle=False,
                         chunksizes=(672,len(ds_out['feature_id'])))

In [ ]:
ds.streamflow.isel(feature_id=[1,2,50000], time=slice(0,672))

In [ ]:
ds_out.streamflow

In [ ]:
encoding

In [ ]:
672*93754*4/1e6

In [ ]:
%%time
ds_out.to_netcdf('subset.nc', mode='w', encoding=encoding)

In [ ]:
10000/2500